### 1. **Initializing Spark Session**:


In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=5099c5a9fe1e9fb3f160fd5339d5a599b952720b9a231400665223f710d685c3
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, MapType
from pyspark.sql.functions import explode, map_keys, col

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

- Imports necessary PySpark libraries.
- Initializes a Spark session with the application name 'SparkByExamples.com'.


### 2. **Defining Sample Data**:


In [5]:
dataDictionary = [
    ('James', {'hair': 'black', 'eye': 'brown'}),
    ('Michael', {'hair': 'brown', 'eye': None}),
    ('Robert', {'hair': 'red', 'eye': 'black'}),
    ('Washington', {'hair': 'grey', 'eye': 'grey'}),
    ('Jefferson', {'hair': 'brown', 'eye': ''})
]

   - Defines sample data with a dictionary column.


### 3. **Creating DataFrame with Inferred Schema**:


In [6]:
df = spark.createDataFrame(data=dataDictionary, schema=['name', 'properties'])
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+----------+-----------------------------+
|name      |properties                   |
+----------+-----------------------------+
|James     |{eye -> brown, hair -> black}|
|Michael   |{eye -> NULL, hair -> brown} |
|Robert    |{eye -> black, hair -> red}  |
|Washington|{eye -> grey, hair -> grey}  |
|Jefferson |{eye -> , hair -> brown}     |
+----------+-----------------------------+




   - Creates a DataFrame with the inferred schema.
   - Prints the schema and shows the DataFrame content.


### 4. **Creating DataFrame with Explicit Schema**:


In [7]:
schema = StructType([
    StructField('name', StringType(), True),
    StructField('properties', MapType(StringType(), StringType()), True)
])

df2 = spark.createDataFrame(data=dataDictionary, schema=schema)
df2.printSchema()
df2.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+----------+-----------------------------+
|name      |properties                   |
+----------+-----------------------------+
|James     |{eye -> brown, hair -> black}|
|Michael   |{eye -> NULL, hair -> brown} |
|Robert    |{eye -> black, hair -> red}  |
|Washington|{eye -> grey, hair -> grey}  |
|Jefferson |{eye -> , hair -> brown}     |
+----------+-----------------------------+



   - Defines an explicit schema using `StructType`.
   - Creates a DataFrame with the explicit schema.
   - Prints the schema and shows the DataFrame content.


### 5. **Converting RDD to DataFrame with Separate Columns**:


In [8]:
df3 = df.rdd.map(lambda x: (x.name, x.properties["hair"], x.properties["eye"])) \
            .toDF(["name", "hair", "eye"])
df3.printSchema()
df3.show()

root
 |-- name: string (nullable = true)
 |-- hair: string (nullable = true)
 |-- eye: string (nullable = true)

+----------+-----+-----+
|      name| hair|  eye|
+----------+-----+-----+
|     James|black|brown|
|   Michael|brown| NULL|
|    Robert|  red|black|
|Washington| grey| grey|
| Jefferson|brown|     |
+----------+-----+-----+



- Converts the DataFrame to an RDD and then maps each row to extract `name`, `hair`, and `eye`.
- Converts the RDD back to a DataFrame with separate columns for `name`, `hair`, and `eye`.
- Prints the schema and shows the new DataFrame content.


### 6. **Extracting Columns from Dictionary Using withColumn and getItem**:


In [9]:
df.withColumn("hair", df.properties.getItem("hair")) \
  .withColumn("eye", df.properties.getItem("eye")) \
  .drop("properties") \
  .show()

+----------+-----+-----+
|      name| hair|  eye|
+----------+-----+-----+
|     James|black|brown|
|   Michael|brown| NULL|
|    Robert|  red|black|
|Washington| grey| grey|
| Jefferson|brown|     |
+----------+-----+-----+



- Uses `withColumn` and `getItem` to create new columns `hair` and `eye` from the `properties` dictionary.
- Drops the original `properties` column.
- Shows the updated DataFrame content.


### 7. **Extracting Columns from Dictionary Using withColumn and Direct Indexing**:


In [10]:
df.withColumn("hair", df.properties["hair"]) \
  .withColumn("eye", df.properties["eye"]) \
  .drop("properties") \
  .show()

+----------+-----+-----+
|      name| hair|  eye|
+----------+-----+-----+
|     James|black|brown|
|   Michael|brown| NULL|
|    Robert|  red|black|
|Washington| grey| grey|
| Jefferson|brown|     |
+----------+-----+-----+



   - Similar to the previous step, but uses direct indexing to extract `hair` and `eye` from the `properties` dictionary.
   - Shows the updated DataFrame content.


### 8. **Extracting Keys from the Map and Creating Separate Columns**:


In [11]:
keysDF = df.select(explode(map_keys(df.properties))).distinct()
keysList = keysDF.rdd.map(lambda x: x[0]).collect()
keyCols = list(map(lambda x: col("properties").getItem(x).alias(str(x)), keysList))
df.select(df.name, *keyCols).show()

+----------+-----+-----+
|      name|  eye| hair|
+----------+-----+-----+
|     James|brown|black|
|   Michael| NULL|brown|
|    Robert|black|  red|
|Washington| grey| grey|
| Jefferson|     |brown|
+----------+-----+-----+



- Extracts the keys from the `properties` map using `map_keys` and `explode`.
- Collects the distinct keys into a list.
- Creates new columns for each key in the `properties` map using `getItem` and `alias`.
- Selects the `name` column and the newly created columns, and shows the updated DataFrame content.